In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, Input
from keras import applications
from keras.layers import AveragePooling2D, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger
from keras.applications.mobilenet import MobileNet
from keras.layers import Conv2D, Convolution2D, MaxPooling2D, ZeroPadding2D, BatchNormalization, Activation
from keras.optimizers import RMSprop
from keras import backend as K
import numpy as np
import pandas as pd
from keras import layers
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.models import load_model

Using TensorFlow backend.


In [2]:
#parameters
classes = 12
image_size = 1500
batch_size= 6
train_total = 3820
validation_total = 930

In [3]:
model_mobilenet = MobileNet(include_top=False, weights = None,input_shape=(image_size,image_size,3), alpha=0.5)
x = model_mobilenet.output
x = Dropout(0.5)(x)
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(classes, activation='softmax')(x)
model = Model(inputs=model_mobilenet.input, outputs=x)
model.compile(loss='categorical_crossentropy',
             optimizer=RMSprop(lr=1e-6),
             metrics=['accuracy'])
model.load_weights('mobilenet_from_scratch_image_1500_local_split.hdf5')

In [4]:
model_mobilenet_2 = MobileNet(include_top=False, weights = None,input_shape=(1000,1000,3), alpha=0.5)
x = model_mobilenet_2.output
x = Dropout(0.5)(x)
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(classes, activation='softmax')(x)
model_2 = Model(inputs=model_mobilenet_2.input, outputs=x)
model_2.load_weights('mobilenet_from_scratch_save_best_image_increase_1000.hdf5')
model_2.compile(loss='categorical_crossentropy',
             optimizer=RMSprop(lr=1e-6),
             metrics=['accuracy'])

In [5]:
model_mobilenet_3 = MobileNet(include_top=False, weights = None,input_shape=(image_size,image_size,3), alpha=0.5)
x = model_mobilenet_3.output
x = Dropout(0.5)(x)
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(2, activation='softmax')(x)
model_3 = Model(inputs=model_mobilenet_3.input, outputs=x)
model_3.compile(loss='categorical_crossentropy',
             optimizer=RMSprop(lr=1e-6),
             metrics=['accuracy'])
model_3.load_weights('11_mobilenet_from_scratch_model_2_v3_increase.hdf5')

In [6]:
model_mobilenet_4 = MobileNet(include_top=False, weights = None,input_shape=(image_size,image_size,3), alpha=0.5)
x = model_mobilenet_4.output
x = Dropout(0.5)(x)
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(classes, activation='softmax')(x)
model_4 = Model(inputs=model_mobilenet_4.input, outputs=x)
model_4.compile(loss='categorical_crossentropy',
             optimizer=RMSprop(lr=1e-6),
             metrics=['accuracy'])
model_4.load_weights('mobilenet_image_1500_local_split_pseudo_label.hdf5')

In [7]:
model_mobilenet_5 = MobileNet(include_top=False, weights = None,input_shape=(1000,1000,3), alpha=0.5)
x = model_mobilenet_5.output
x = MaxPooling2D((4,4),strides=(4,4))(x)
x = Flatten()(x)
x = Dense(4096, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(classes, activation='softmax')(x)
model_5 = Model(inputs=model_mobilenet_5.input, outputs=x)
model_5.compile(loss='categorical_crossentropy',
             optimizer=RMSprop(lr=1e-6),
             metrics=['accuracy'])
model_5.load_weights('mobilenet_fixed_bottom_image_1500_local_split.hdf5')

In [8]:
model_mobilenet_6 = MobileNet(include_top=False, weights = None,input_shape=(image_size,image_size,3), alpha=0.5)
x = model_mobilenet_6.output
x = Dropout(0.5)(x)
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(2, activation='softmax')(x)
model_6 = Model(inputs=model_mobilenet_6.input, outputs=x)
model_6.compile(loss='categorical_crossentropy',
             optimizer=RMSprop(lr=1e-6),
             metrics=['accuracy'])
model_6.load_weights('11_mobilenet_from_scratch_model_2_v3_increase_second_split.hdf5')

In [9]:
model_mobilenet_7 = MobileNet(include_top=False, weights = None,input_shape=(image_size,image_size,3), alpha=0.5)
x = model_mobilenet_7.output
x = Dropout(0.5)(x)
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(2, activation='softmax')(x)
model_7 = Model(inputs=model_mobilenet_7.input, outputs=x)
model_7.compile(loss='categorical_crossentropy',
             optimizer=RMSprop(lr=1e-6),
             metrics=['accuracy'])

In [10]:
len(model_3.layers[2].get_weights())

4

In [11]:
for layer1,layer2,layer3 in zip(model_7.layers,model_6.layers,model_3.layers):
    layer1.set_weights([(i+j)/2 for i,j in zip(layer2.get_weights(),layer3.get_weights())])

In [12]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range = 0.2,
                                   rotation_range = 45,
                                   height_shift_range=0.2,
                                   width_shift_range=0.2,
                                   fill_mode='reflect',
                                   horizontal_flip=True,
                                   vertical_flip=True)
validation_datagen = ImageDataGenerator(rescale = 1./255)
train_generator = train_datagen.flow_from_directory(
                    'train',
                    target_size=(image_size,image_size),
                    batch_size=batch_size,
                    class_mode='categorical',
                    shuffle=True
                    )
validation_generator = validation_datagen.flow_from_directory(
                        'validation',
                        target_size=(image_size,image_size),
                        batch_size=batch_size,
                        class_mode='categorical',
                        shuffle=False)

Found 3820 images belonging to 12 classes.
Found 930 images belonging to 12 classes.


In [13]:
train_datagen_2 = ImageDataGenerator(rescale=1./255,
                                   shear_range = 0.2,
                                   rotation_range = 90,
                                   height_shift_range=0.2,
                                   width_shift_range=0.2,
                                   zoom_range=0.2,
                                   fill_mode='reflect',
                                   horizontal_flip=True,
                                   vertical_flip=True)
validation_datagen_2 = ImageDataGenerator(rescale = 1./255)
train_generator_2 = train_datagen_2.flow_from_directory(
                    'train_2',
                    target_size=(image_size,image_size),
                    batch_size=batch_size,
                    class_mode='categorical',
                    shuffle=True
                    )
validation_generator_2 = validation_datagen_2.flow_from_directory(
                        'validation_2',
                        target_size=(image_size,image_size),
                        batch_size=batch_size,
                        class_mode='categorical',
                        shuffle=False)

Found 739 images belonging to 2 classes.
Found 224 images belonging to 2 classes.


In [14]:
model_3.evaluate_generator(validation_generator_2, int(np.ceil(224/batch_size)))

[0.1626936701712631, 0.95535713753529961]

In [15]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_generator = test_datagen.flow_from_directory(
                    'test',
                    shuffle=False,
                    target_size=(image_size,image_size),
                    batch_size=batch_size,
                    class_mode=None)


Found 794 images belonging to 1 classes.


In [16]:
predictions = model.predict_generator(test_generator,int(np.ceil(794/batch_size)))
class_ids = {train_generator.class_indices[x]: x for x in train_generator.class_indices}
predicted_classes = [class_ids[x] for x in np.argmax(predictions, axis=1)]


In [17]:
predictions_7 = model_7.predict_generator(test_generator,int(np.ceil(794/batch_size)))
class_ids = {train_generator_2.class_indices[x]: x for x in train_generator_2.class_indices}
predicted_classes_7 = [class_ids[x] for x in np.argmax(predictions_7, axis=1)]


In [18]:
predictions_3 = model_3.predict_generator(test_generator,int(np.ceil(794/batch_size)))
class_ids = {train_generator_2.class_indices[x]: x for x in train_generator_2.class_indices}
predicted_classes_3 = [class_ids[x] for x in np.argmax(predictions_3, axis=1)]


In [19]:
predictions_4 = model_4.predict_generator(test_generator,int(np.ceil(794/batch_size)))
class_ids = {train_generator.class_indices[x]: x for x in train_generator.class_indices}
predicted_classes_4 = [class_ids[x] for x in np.argmax(predictions_4, axis=1)]


In [20]:
predictions_6 = model_6.predict_generator(test_generator,int(np.ceil(794/batch_size)))
class_ids = {train_generator_2.class_indices[x]: x for x in train_generator_2.class_indices}
predicted_classes_6 = [class_ids[x] for x in np.argmax(predictions_6, axis=1)]


In [21]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_generator_1000 = test_datagen.flow_from_directory(
                    'test',
                    shuffle=False,
                    target_size=(1000,1000),
                    batch_size=batch_size,
                    class_mode=None)


Found 794 images belonging to 1 classes.


In [22]:
predictions_2 = model_2.predict_generator(test_generator_1000,int(np.ceil(794/batch_size)))
class_ids = {train_generator.class_indices[x]: x for x in train_generator.class_indices}
predicted_classes_2 = [class_ids[x] for x in np.argmax(predictions_2, axis=1)]


In [23]:
predictions_5 = model_5.predict_generator(test_generator_1000,int(np.ceil(794/batch_size)))
class_ids = {train_generator.class_indices[x]: x for x in train_generator.class_indices}
predicted_classes_5 = [class_ids[x] for x in np.argmax(predictions_5, axis=1)]


In [54]:
predictions_max = np.maximum(predictions,predictions_2)

In [55]:
class_ids = {train_generator.class_indices[x]: x for x in train_generator.class_indices}
predicted_classes_max = [class_ids[x] for x in np.argmax(predictions_max, axis=1)]

In [56]:
predictions_bottom_model = (predictions + predictions_2)/2

In [57]:
class_ids = {train_generator.class_indices[x]: x for x in train_generator.class_indices}
predicted_classes_bottom_model = [class_ids[x] for x in np.argmax(predictions_bottom_model, axis=1)]

In [58]:
indexes = []
for i in range(len(np.argmax(predictions_bottom_model, axis=1))):
    if((np.argmax(predictions_bottom_model, axis=1)[i]==0) or (np.argmax(predictions_bottom_model, axis=1)[i]==6)):
        indexes.append(i)

In [59]:
len(indexes)

159

In [103]:
binary_predictions_averaged = np.maximum(predictions_6,predictions_3)

In [60]:
for i in indexes:
    predictions_bottom_model[i][0] = (predictions_bottom_model[i][0] + predictions_3[i][0])/2
    predictions_bottom_model[i][6] = (predictions_bottom_model[i][6] + predictions_3[i][1])/2

In [61]:
class_ids = {train_generator.class_indices[x]: x for x in train_generator.class_indices}
predicted_classes_bottom_model = [class_ids[x] for x in np.argmax(predictions_bottom_model, axis=1)]

In [62]:
for i in indexes:
    predictions_max[i][0] = np.maximum(predictions_max[i][0],predictions_7[i][0])
    predictions_max[i][6] = np.maximum(predictions_max[i][6],predictions_7[i][1])

In [63]:
class_ids = {train_generator.class_indices[x]: x for x in train_generator.class_indices}
predicted_classes_max = [class_ids[x] for x in np.argmax(predictions_max, axis=1)]

In [64]:
for i in indexes:
    predictions_bottom_model[i][0] = (predictions_bottom_model[i][0] + predictions_max[i][0])/2
    predictions_bottom_model[i][6] = (predictions_bottom_model[i][6] + predictions_max[i][6])/2

In [65]:
class_ids = {train_generator.class_indices[x]: x for x in train_generator.class_indices}
predicted_classes_bottom_model = [class_ids[x] for x in np.argmax(predictions_bottom_model, axis=1)]

In [51]:
for i in indexes:
    predictions_max[i][0] = np.maximum(predictions_max[i][0],predictions_bottom_model[i][0])
    predictions_max[i][6] = np.maximum(predictions_max[i][6],predictions_bottom_model[i][6])

In [52]:
class_ids = {train_generator.class_indices[x]: x for x in train_generator.class_indices}
predicted_classes_max = [class_ids[x] for x in np.argmax(predictions_max, axis=1)]

In [66]:
for i in indexes:
    if(predicted_classes_bottom_model[i]!=predicted_classes_max[i]):
        print(i)

165
688


In [67]:
from os.path import basename
test_ids = [basename(x) for x in test_generator.filenames]
submission = pd.DataFrame({'file':test_ids,'species':predicted_classes_max})
submission.to_csv('submission_mobilenet_max_predictions_binary_weights_averaged_average_of_two_ensembles.csv', encoding="utf8", index=False)

In [68]:
from IPython.display import FileLink
FileLink('submission_mobilenet_max_predictions_binary_weights_averaged_average_of_two_ensembles.csv')

/home/sainath/plant_seeding/submission_mobilenet_max_predictions_binary_weights_averaged_average_of_two_ensembles.csv